#### Model 3: oracle for TIN (tax identification number) confirmation. Should get verified TINs from the tax authorities (helps to resolve last mile problem to check valid TINs).

In [37]:
import json
import web3

from web3 import Web3
from solc import compile_source
from web3.contract import ConciseContract

In [38]:
# Solidity source code
#example based on https://blockgeeks.com/guides/solidity/
oracle_source_code = '''
pragma solidity ^0.4.21;

//Oracle to confirm Tax Residence Sertificates
contract Oracle 
    { struct Request {
        bytes data;
        function(uint) external callback; 
    }

    Request[] requests;
    
    event NewRequest(uint);
    
    function query(bytes memory data, function(uint) external callback) public {
        requests.push(Request(data, callback));
        emit NewRequest(requests.length - 1); 
    }
    
    function reply(uint requestID, uint response) public {
        requests[requestID].callback(response);
    } 
    
    address Admin;
    
    //Trying to solve last mile problem - images schould be verifyed
    mapping ( bytes32 => verifiedImage) verifiedImages;
    bytes32[] imagesByVerificatorHash; 
    
    //Or last mile may be put on users - if they want to be in oracle database - they should verify their data
    mapping ( address => User ) Users;
    address[] usersByAddress;
    
    struct verifiedImage {
        string imageURL;
        uint timeStamp;
    }
    
    struct User {
        string name;
        string city;
        string state;
        string country;
        bytes32[] certificateImages;
    }
    
    constructor () public payable {  
        Admin = msg.sender;
    }
    
    modifier onlyAdmin() {
      if (msg.sender != Admin)
        revert('No rights');
      // Do not forget the "_;"! It will be replaced by the actual function body when the modifier is used.
      _;
    }
    
    function removeUser(address badUser) public onlyAdmin returns (bool success) {
        delete Users[badUser];
        return true;
    }
    
    function removeImage(bytes32 badImage) public onlyAdmin returns (bool success) {
        delete verifiedImages[badImage];
        return true;
    }
    
    function registerNewUser(string memory name, 
        string memory city, string memory state, string memory country) public returns (bool success) {
        address thisNewAddress = msg.sender;
  
        if(bytes(Users[msg.sender].name).length == 0 && bytes(name).length != 0){
            Users[thisNewAddress].name = name;
            Users[thisNewAddress].city = city;
            Users[thisNewAddress].state = state;
            Users[thisNewAddress].country = country;
            usersByAddress.push(thisNewAddress);
            return true;
        } else {
            return false;
        }
    }
  
  function addImageToUser(string memory imageURL,bytes32 SHA256Hash) public returns (bool success) {
    address thisNewAddress = msg.sender;
    if(bytes(Users[thisNewAddress].name).length != 0){ 
      if(bytes(imageURL).length != 0){   // ) {  
        
        if(bytes(verifiedImages[SHA256Hash].imageURL).length == 0) {
          imagesByVerificatorHash.push(SHA256Hash);
        }
        verifiedImages[SHA256Hash].imageURL = imageURL;
        verifiedImages[SHA256Hash].timeStamp = block.timestamp; 
        Users[thisNewAddress].certificateImages.push(SHA256Hash); 
        return true;
      } else {
        return false; 
      }
      return true;
    } else {
      return false; 
    }
  }

    function getUsers() view public returns (address[] memory) 
        { return usersByAddress; }
    
    function getUser(address userAddress) view public
        returns (string memory,string memory,string memory,string memory,bytes32[] memory) 
    {
        return (Users[userAddress].name,Users[userAddress].city,Users[userAddress].state,Users[userAddress].country,Users[userAddress].certificateImages);
    }
  
    function getAllImages() view public returns (bytes32[] memory) 
    { return imagesByVerificatorHash; }
  
    function getUserImages(address userAddress) view public returns (bytes32[] memory) 
        { return Users[userAddress].certificateImages; }

    function getImage(bytes32 SHA256Hash) view public returns (string memory,uint) {
        return (verifiedImages[SHA256Hash].imageURL,verifiedImages[SHA256Hash].timeStamp);
  }
}
'''

In [39]:
# Solidity Compiler

compiled_sol = compile_source(oracle_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:Oracle']

In [40]:
# web3.py instance
w3 = Web3(Web3.EthereumTesterProvider())

In [41]:
# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [42]:
# Instantiate and deploy contract
ContractDeploy = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

In [43]:
# Submit the transaction that deploys the contract
tx_hash = ContractDeploy.constructor().transact()

In [44]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [45]:
# Create the contract instance with the newly-deployed address
contract_inst = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [46]:
# Perform a transaction within a "Blockchain"

tx_hash = contract_inst.functions.registerNewUser('APIAP', 'NSW','Parramatta','Australia').transact()

# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x5979aea0ec919ec5dc18f866d45a390cafa4b16e35084220730ef13aa6bf92b7'),
 'transactionIndex': 0,
 'blockNumber': 2,
 'blockHash': HexBytes('0x0c1f01018133134def2a7d9b672d2bc7a3a6976113e8aae104b6e8ce0e6abb73'),
 'cumulativeGasUsed': 149838,
 'gasUsed': 149838,
 'contractAddress': None,
 'logs': [],
 'status': 1})

In [47]:
# Perform a transaction within a "Blockchain"

tx_hash = contract_inst.functions.addImageToUser('www.apricot.net/docs/Certificate-of-Tax-Residency.jpg',
                                                 '0x6c3e007e281f6948b37c511a11e43c8026d2a16a8a45fed4e83379b66b0ab927').transact()

# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x42de0537d16ad206bffc06bb24b447dfb4b0e6785ab14aef5dba2d4bd36b8350'),
 'transactionIndex': 0,
 'blockNumber': 3,
 'blockHash': HexBytes('0xc18a15380d7d1eabafe73a6d6bdcd1158f9d5d4f933aa1240cba86b7d591fac4'),
 'cumulativeGasUsed': 190517,
 'gasUsed': 190517,
 'contractAddress': None,
 'logs': [],
 'status': 1})

In [48]:
# Perform a transaction within a "Blockchain"

tx_hash = contract_inst.functions.getImage('0x6c3e007e281f6948b37c511a11e43c8026d2a16a8a45fed4e83379b66b0ab927').transact()

# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0xbaf715fdb83e3b5c1af891be4ec32e81daedbc6f4a25c7e4dc420053e41f25ac'),
 'transactionIndex': 0,
 'blockNumber': 4,
 'blockHash': HexBytes('0x32af966a91cdcacaa845f75e187ce86469f101b2630e5edecad3389d1d7dc86c'),
 'cumulativeGasUsed': 25766,
 'gasUsed': 25766,
 'contractAddress': None,
 'logs': [],
 'status': 1})

In [49]:
user_source_code = '''
pragma solidity >=0.4.22 <0.6.0;


contract OracleUser {

    address oracleAddr;
    
    function ExistingOracle(address _t) public {
        oracleAddr = _t;
    }

    function oracleQuery () public {
        
        oracleAddr.call(bytes4(keccak256("getUsers()")));

    }
}
'''